In [1]:
star_params_file = 'stellar_params_CTL.csv'

In [ ]:
from random import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightkurve as lk
import batman
from transitleastsquares import transitleastsquares
import seaborn as sns
import os
from functools import lru_cache
from itertools import product
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from itertools import groupby
from operator import itemgetter

# ------------------------------------
# 1. Cached light curve loader
# ------------------------------------
@lru_cache(maxsize=10)
def get_lightcurve(tic):
    lc_collection = lk.search_lightcurve(tic, mission="TESS", author="TESS-SPOC", cadence="long").download_all(quality_bitmask="hard")
    if lc_collection is None or len(lc_collection) == 0:
        return None
    
    lc = lc_collection.stitch().remove_nans()

    #plt.plot(lc.time.value, lc.flux.value, 'k.', markersize=1, alpha=0.5)


    return lc.to_pandas().reset_index()

# ------------------------------------
# 2. Flare masking: ±3hr from >5σ events
# ------------------------------------
def mask_flares(df):
    median = df['flux'].median()
    std = df['flux'].std()
    threshold = median + 5 * std
    flare_times = df.loc[df['flux'] > threshold, 'time'].values

    def within_3hr(t):
        return np.any(np.abs(flare_times - t) <= 0.125)

    flare_mask = df['time'].apply(within_3hr)
    return df[~flare_mask].reset_index(drop=True)

# ------------------------------------
# 3. Inject transit only (returns multiplier)
# ------------------------------------
def inject_model(time, period_bin, rp_earth_bin, st_radius, st_mass, st_teff):
    # Constants
    R_earth = 6.371e6        # meters
    R_sun = 6.957e8          # meters
    M_sun = 1.989e30         # kg
    G = 6.67430e-11          # m^3 / kg / s^2

    #np.random.seed(2)
    rp_earth = np.random.uniform(rp_earth_bin[0], rp_earth_bin[1])
    period = np.random.uniform(period_bin[0], period_bin[1])
    print(f"Injected planet radius (Earth Radii): {rp_earth:.2f}, period (days): {period:.2f}")

    # Convert radius: Earth radii to stellar radii
    rp_rstar = (rp_earth * R_earth) / (st_radius * R_sun)

    # Convert mass to kg and period to seconds
    period_sec = period * 86400
    st_mass_kg = st_mass * M_sun
    st_radius_m = st_radius * R_sun

    # Compute semi-major axis in meters
    a_meters = ((G * st_mass_kg * period_sec**2) / (4 * np.pi**2))**(1 / 3)

    # Convert semi-major axis to stellar radii
    a_rstar = a_meters / st_radius_m

    impact_param_upper = .9
    impact_param_lower = 0
    inclination_upper = np.degrees(np.arccos(impact_param_upper * st_radius_m / a_meters))
    inclination_lower = np.degrees(np.arccos(impact_param_lower * st_radius_m / a_meters))
    print(f"Inclination range: {inclination_lower:.2f} to {inclination_upper:.2f} degrees")
    #randomized inclination
    inclination =np.random.uniform(inclination_lower, inclination_upper)
    print(f"rp/rstar^2: {rp_rstar**2:.6f}, a/rstar: {a_rstar:.2f}, period: {period:.2f} days, inclination: {inclination:.2f} degrees")


    #LD LAWS from limb_darkening_laws_investigation.ipynb
    # Quadratic limb darkening coefficients from provided table
    if st_teff < 3500:
        u1, u2 = 0.1598, 0.4534
    elif 3500 <= st_teff < 4000:
        u1, u2 = 0.2366, 0.3750
    elif 4000 <= st_teff < 4500:
        u1, u2 = 0.4540, 0.2136
    elif 4500 <= st_teff < 5000:
        u1, u2 = 0.4672, 0.1826
    elif 5000 <= st_teff < 5500:
        u1, u2 = 0.4068, 0.2063
    elif 5500 <= st_teff < 6000:
        u1, u2 = 0.3632, 0.2209
    elif 6000 <= st_teff < 6500:
        u1, u2 = 0.3273, 0.2323
    elif 6500 <= st_teff < 7000:
        u1, u2 = 0.3101, 0.2284
    elif 7000 <= st_teff:
        u1, u2 = 0.2747, 0.2456

        

    t0 =  np.random.uniform(0, period)
    #t0 = np.random.uniform(time.min() + 1, time.max() - 1)


    # Set up transit parameters
    mparams = batman.TransitParams()
    mparams.t0 = t0
    mparams.per = period
    mparams.rp = rp_rstar
    mparams.a = a_rstar
    mparams.inc = inclination
    mparams.ecc = 0
    mparams.w = 90
    mparams.u = [u1, u2]
    mparams.limb_dark = "quadratic"

    model = batman.TransitModel(mparams, time)

    return model.light_curve(mparams), t0, inclination, rp_earth, period



# ------------------------------------
# 5. Run detection across grid 
# ------------------------------------
def run_detection_for_tic(star_params_csv, tic, periods_bins, radii_bins, output_folder, plots=False, xlim=None):
    
    lc = get_lightcurve(tic)
    if lc is None:
        print(f"❌ No light curve for {tic}")
        return pd.DataFrame()

    base_df = lc[['time', 'flux']].copy()
    

    raw_time = base_df['time'].values
    raw_flux = base_df['flux'].values
    if plots == True:
        plt.plot(raw_time, raw_flux, 'k.', markersize=1, alpha=0.5)
        plt.title(f"Bare Lightcurve for {tic}")
        if xlim is not None:
            plt.xlim(xlim)
        plt.show()
    
   

    # Get stellar parameters
    tic_id = int(tic.split()[1])
    star_params = pd.read_csv(star_params_csv).query(f"id == {tic_id}")
    if star_params.empty:
        print(f"⚠️ No stellar params for {tic}")
        return pd.DataFrame()

    st_radius = star_params['RAD'].values[0]
    st_mass = star_params['MASS'].values[0] 
    st_teff = star_params['teff'].values[0]
    st_tmag = star_params['tmag'].values[0]

    results = pd.DataFrame()
     # Loop over period-radius grid
    for period_bin, rp_earth_bin in product(periods_bins, radii_bins):
        detections = 0
        for trial in range(trials):
            print(f"Running trial {trial + 1}/{trials} for {tic}, Period: {period_bin}, Rp (Earth Radii): {rp_earth_bin}")
            # plt.plot(raw_time, raw_flux, 'k.', markersize=1, alpha=0.5)
            # plt.title(f"Bare Lightcurve for {tic}")
            # #plt.xlim(1650,1700)
            # plt.show()
            print(tic)
            transit_signal, true_t0, true_inc, true_rp, true_period = inject_model(raw_time, period_bin, rp_earth_bin, st_radius, st_mass, st_teff)
            injected_flux = raw_flux * transit_signal
            injected_flux_mean_normalized = injected_flux #/ np.mean(injected_flux)
            injected_df_mean_normalized = pd.DataFrame({'time': raw_time, 'flux': injected_flux_mean_normalized})
            flux_normalized_flares_removed = mask_flares(injected_df_mean_normalized)

            if plots == True:
                plt.plot(flux_normalized_flares_removed['time'], flux_normalized_flares_removed['flux'], 'k.', markersize=1, alpha=0.5)
                plt.title(f"Injected Lightcurve for tic {tic} unflattened")
                if xlim is not None:
                    plt.xlim(xlim)
                plt.show()

            lc = lk.LightCurve(time=flux_normalized_flares_removed['time'], flux=flux_normalized_flares_removed['flux'])
            time = lc.time.value  # in days
            if len(time) < 2:
                return None

            cadence_days = time[1] - time[0]
            duration_days = 2
            wl = int(np.round(duration_days / cadence_days))
            if wl % 2 == 0:
                wl += 1  # make it odd, as required

            #print(f"Cadence: {cadence_days:.5f} days, window_length: {wl}")

            lc_flat = lc.flatten(window_length=wl, polyorder=2)
            flat_df = lc_flat.to_pandas().reset_index()
            time = flat_df['time'].values
            flux = flat_df['flux'].values
            #flux = flux / np.median(flux)
            if plots == True:
                plt.plot(time, flux, 'k.', markersize=1, alpha=0.5)
                plt.title(f"Injected Lightcurve for {tic} flattened")
                if xlim is not None:
                    plt.xlim(xlim)
                plt.show()

            lightcurve_data = pd.DataFrame({'time': time, 'flux': flux, 'true_t0': true_t0, 'true_inc': true_inc, 'true_rp_earth': true_rp, 'true_period': true_period})
            file_name = f'{output_folder}/P{true_period:.3f}_Rp{true_rp:.3f}_trial{trial}.csv'
            os.makedirs(output_folder, exist_ok=True)
            lightcurve_data.to_csv(file_name, index=False)

            

# ------------------------------------
# 7. Main driver
# ------------------------------------
if __name__ == "__main__":


    # Load TIC IDs from the CSV file
    star_params = pd.read_csv(star_params_file)



    df_save = pd.DataFrame()

    
for _, row in star_params.iterrows():

    tic = f"TIC {int(row['id'])}"


    #print(tic)
    output_folder = f'./3_import_to_geryon/injected_flattened_lcs/{tic.replace(" ", "_")}'
    if os.path.exists(output_folder):
        print(f"Skipping {tic}, already processed.")
        continue

    ## DANA NOTES, you can modify the period/radius bins here

    # Log-spaced list between 1.4 and 5 (inclusive), in log10 space
    radii_centers = np.logspace(np.log10(1.4), np.log10(5), num=6)
    # Log-spaced list between 0.7 and 25 (inclusive), in log10 space
    periods_centers = np.logspace(np.log10(0.7), np.log10(25), num=7)

    # Convert radii_centers and periods_centers to floats with up to 3 decimals
    radii_centers = [float(f"{v:.3f}") for v in radii_centers]
    periods_centers = [float(f"{v:.3f}") for v in periods_centers]
    

    # Generate bin edges from centers
    periods_log = np.log10(periods_centers)
    log_step = periods_log[1] - periods_log[0]
    period_bin_edges = [10**(periods_log[0] - log_step/2)]
    period_bin_edges.extend([10**((periods_log[i] + periods_log[i+1])/2) for i in range(len(periods_log)-1)])
    period_bin_edges.append(10**(periods_log[-1] + log_step/2))
    periods_bins = [(period_bin_edges[i], period_bin_edges[i+1]) for i in range(len(period_bin_edges)-1)]

    radii_log = np.log10(radii_centers)
    log_step = radii_log[1] - radii_log[0]
    radius_bin_edges = [10**(radii_log[0] - log_step/2)]
    radius_bin_edges.extend([10**((radii_log[i] + radii_log[i+1])/2) for i in range(len(radii_log)-1)])
    radius_bin_edges.append(10**(radii_log[-1] + log_step/2))
    radii_bins = [(radius_bin_edges[i], radius_bin_edges[i+1]) for i in range(len(radius_bin_edges)-1)]

    print("Period bins:", periods_bins)
    print("Radius bins:", radii_bins)

    trials = 1


    try:
        df_results = run_detection_for_tic(star_params_file, tic, periods_bins, radii_bins, output_folder, plots=False, xlim=None)
        #save_heatmap(df_results, tic)

    except Exception as e:
        print(f"❌ Error occurred for {tic}: {e}")



Period bins: [(0.5196910044992892, 0.9428679653058534), (0.9428679653058534, 1.7109500284929424), (1.7109500284929424, 3.1051272115647697), (3.1051272115647697, 5.6349936113539645), (5.6349936113539645, 10.226124192478789), (10.226124192478789, 18.55801713545928), (18.55801713545928, 33.673855903780485)]
Radius bins: [(1.2326312688558734, 1.5900943368240765), (1.5900943368240765, 2.0513361499276512), (2.0513361499276512, 2.6460631133818406), (2.6460631133818406, 3.412825808622526), (3.412825808622526, 4.402272140611028), (4.402272140611028, 5.6789083458002745)]


Running trial 1/1 for TIC 288314668, Period: (0.5196910044992892, 0.9428679653058534), Rp (Earth Radii): (1.2326312688558734, 1.5900943368240765)
TIC 288314668
Injected planet radius (Earth Radii): 1.54, period (days): 0.86
Inclination range: 90.00 to 85.17 degrees
rp/rstar^2: 0.005172, a/rstar: 10.69, period: 0.86 days, inclination: 86.30 degrees
Running trial 1/1 for TIC 288314668, Period: (0.5196910044992892, 0.9428679653058534), Rp (Earth Radii): (1.5900943368240765, 2.0513361499276512)
TIC 288314668
Injected planet radius (Earth Radii): 1.74, period (days): 0.84
Inclination range: 90.00 to 85.07 degrees
rp/rstar^2: 0.006622, a/rstar: 10.47, period: 0.84 days, inclination: 87.56 degrees
Running trial 1/1 for TIC 288314668, Period: (0.5196910044992892, 0.9428679653058534), Rp (Earth Radii): (2.0513361499276512, 2.6460631133818406)
TIC 288314668
Injected planet radius (Earth Radii): 2.14, period (days): 0.68
Inclination range: 90.00 to 84.31 degrees
rp/rstar^2: 0.009956, a/rstar: 9.0

Running trial 1/1 for TIC 92994182, Period: (0.5196910044992892, 0.9428679653058534), Rp (Earth Radii): (1.2326312688558734, 1.5900943368240765)
TIC 92994182
Injected planet radius (Earth Radii): 1.48, period (days): 0.84
Inclination range: 90.00 to 84.83 degrees
rp/rstar^2: 0.004069, a/rstar: 9.98, period: 0.84 days, inclination: 89.18 degrees
Running trial 1/1 for TIC 92994182, Period: (0.5196910044992892, 0.9428679653058534), Rp (Earth Radii): (1.5900943368240765, 2.0513361499276512)
TIC 92994182
Injected planet radius (Earth Radii): 1.83, period (days): 0.61
Inclination range: 90.00 to 83.61 degrees
rp/rstar^2: 0.006235, a/rstar: 8.08, period: 0.61 days, inclination: 87.35 degrees
Running trial 1/1 for TIC 92994182, Period: (0.5196910044992892, 0.9428679653058534), Rp (Earth Radii): (2.0513361499276512, 2.6460631133818406)
TIC 92994182
Injected planet radius (Earth Radii): 2.22, period (days): 0.64
Inclination range: 90.00 to 83.80 degrees
rp/rstar^2: 0.009221, a/rstar: 8.33, perio

KeyboardInterrupt: 

In [ ]:
#CHECK THAT EACH FOLDER HAS THE NUMBER OF FILES THAT YOU HAVE CELLS / THE EXPECTED NUMBER OF FILES
from collections import Counter
import os

folder_counts = []
base_dir = './injected_flattened_lcs'

for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        num_files = len(os.listdir(folder_path))
        folder_counts.append(num_files)

count_summary = Counter(folder_counts)
for num_files, num_folders in sorted(count_summary.items()):
    print(f"{num_folders} folders have {num_files} files")

# Print folders that don't have 42 files
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        num_files = len(os.listdir(folder_path))
        if num_files != 42:
            print(f"{folder}: {num_files} files")

FileNotFoundError: [Errno 2] No such file or directory: './injected_flattened_lcs_fixed'